In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

Path = '/content/drive/MyDrive/MLT+Dataset/Dataset/'

In [4]:
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import os
from nltk.stem import PorterStemmer

In [15]:

# Load each English and German file
german_files = ["commoncrawl_de_en.txt", "europarl-v7_de_en.txt", "news-commentary-v9_de_en.txt"]
english_files = ["commoncrawl_en_de.txt", "europarl-v7_en_de.txt", "news-commentary-v9_en_de.txt"]

In [16]:

# Read English and German text files
with open(Path + 'commoncrawl_en_de.txt', 'r', encoding='utf-8') as eng_file:
    english_sentences = eng_file.readlines()

with open(Path + 'commoncrawl_de_en.txt', 'r', encoding='utf-8') as ger_file:
    german_sentences = ger_file.readlines()

# Strip any unnecessary whitespace (e.g., newline characters)
commoncrawl_english_sentences = [sentence.strip() for sentence in english_sentences]
commoncrawl_german_sentences = [sentence.strip() for sentence in german_sentences]

In [17]:
if len(commoncrawl_english_sentences) != len(commoncrawl_german_sentences):
    raise ValueError("The number of sentences in the English and German files do not match.")

In [18]:
commoncrawl_en_de_df = pd.DataFrame({
    'English': commoncrawl_english_sentences,
    'German': commoncrawl_german_sentences
})

In [19]:
commoncrawl_en_de_df.shape

(2399123, 2)

In [20]:
# Read English and German text files
with open(Path + 'europarl-v7_en_de.txt', 'r', encoding='utf-8') as eng_file:
    english_sentences = eng_file.readlines()

with open(Path + 'europarl-v7_de_en.txt', 'r', encoding='utf-8') as ger_file:
    german_sentences = ger_file.readlines()

# Strip any unnecessary whitespace (e.g., newline characters)
europarl_english_sentences = [sentence.strip() for sentence in english_sentences]
europarl_german_sentences = [sentence.strip() for sentence in german_sentences]

In [21]:
if len(europarl_english_sentences) != len(europarl_german_sentences):
    raise ValueError("The number of sentences in the English and German files do not match.")

In [22]:

europarl_en_de_df = pd.DataFrame({
    'English': europarl_english_sentences,
    'German': europarl_english_sentences
})

In [23]:
europarl_en_de_df.shape

(1920209, 2)

In [13]:
# commentary_english_sentences = [sentence.strip() for sentence in english_sentences if sentence.strip()]
# commentary_german_sentences = [sentence.strip() for sentence in german_sentences if sentence.strip()]

In [24]:
# Read English and German text files
# cleaning data for news comm in german
def is_special_characters(line):
    return re.match(r'^[^a-zA-Z0-9\s]+$', line) is not None

commentary_english_sentences = []
commentary_german_sentences = []

with open(Path + 'news-commentary-v9_en_de.txt', 'r', encoding='utf-8') as file:
  for line in file:
        cleaned_line = line.strip()

        if cleaned_line and not cleaned_line.isdigit() and not is_special_characters(cleaned_line):
            # Replace multiple spaces with a single space
            cleaned_line = re.sub(r'\s+', ' ', cleaned_line)
            commentary_english_sentences.append(cleaned_line)

with open(Path + 'news-commentary-v9_en_de.txt', 'r', encoding='utf-8') as file:
  for line in file:
        cleaned_line = line.strip()

        if cleaned_line and not cleaned_line.isdigit() and not is_special_characters(cleaned_line):
            # Replace multiple spaces with a single space
            cleaned_line = re.sub(r'\s+', ' ', cleaned_line)
            commentary_german_sentences.append(cleaned_line)

In [25]:
if len(commentary_english_sentences) != len(commentary_german_sentences):
    raise ValueError("The number of sentences in the English and German files do not match.")

In [26]:
commentary_en_de_df = pd.DataFrame({
    'English': commentary_english_sentences,
    'German': commentary_german_sentences
})

In [27]:
commentary_en_de_df.shape

(201553, 2)

In [64]:
final_df = pd.concat([commoncrawl_en_de_df, europarl_en_de_df, commentary_en_de_df], axis=0, ignore_index=True)

In [59]:
final_df.shape

(4520885, 2)

In [47]:
500000/4520885

0.1105978143659925

In [ ]:
# Cleaning

In [65]:
train_df = final_df.sample(frac=0.011, random_state=12)
hold_out_df = final_df.sample(frac=0.01, random_state=22)

In [66]:
final_df.shape

(4520885, 2)

In [67]:
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters and punctuation (except apostrophes in contractions)
    text = re.sub(r"[^a-zA-ZäöüßÄÖÜéèàùâêîôûç'\s]", '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning function to both columns
train_df['English'] = train_df['English'].apply(clean_text)
train_df['German'] = train_df['German'].apply(clean_text)

# Display cleaned DataFrame
print(train_df)

                                                   English  \
2067265           the counterpart appeared two years later   
152051             public parking is available metres away   
1684613  we guarantee the return of amount paid for the...   
2757674  that question was put to the citizens of switz...   
3663922  we welcome that because we the united states n...   
...                                                    ...   
2177426  since the beginning of they have been collabor...   
3777036  standardised eu regulations have made crossbor...   
622707   without covering the dish and contents with th...   
883410   use the yellow keys on the left to indicate yo...   
425100   there is a common misconception that the smili...   

                                                    German  
2067265         das gegenstück erscheint zwei jahre später  
152051   öffentliche parkmöglichkeiten befinden sich in...  
1684613  wir garantieren die zurückzahlung der für die ...  
2757674  th

In [68]:
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters and punctuation (except apostrophes in contractions)
    text = re.sub(r"[^a-zA-ZäöüßÄÖÜéèàùâêîôûç'\s]", '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning function to both columns
hold_out_df['English'] = hold_out_df['English'].apply(clean_text)
hold_out_df['German'] = hold_out_df['German'].apply(clean_text)

# Display cleaned DataFrame
print(hold_out_df)

                                                   English  \
2703086  the commission has offered to cooperate in the...   
971216   these persons can also subscribe to inquiries ...   
1810753  um so wichtiger und schoner ist die realtime p...   
4465035  on balance that may even make membership more ...   
343285   excellent clean nice neighbourhood near the be...   
...                                                    ...   
207572   wherefore be faithful stand in the office whic...   
1279568        thank you for your interest in our products   
2761868           i shall stop the list there mr president   
93433    room service meetingbanquet facilities airport...   
4404829               so why do they have any rights to it   

                                                    German  
2703086  the commission has offered to cooperate in the...  
971216   sie können anfragen von journalisten abonniere...  
1810753  um so wichtiger und schöner ist die realtime p...  
4465035  on

In [69]:
train_df = train_df.reset_index()
hold_out_df = hold_out_df.reset_index()

In [34]:
# Preprocess

In [70]:
train_df.head()

,index,English,German
0,2067265,the counterpart appeared two years later,das gegenstück erscheint zwei jahre später
1,152051,public parking is available metres away,öffentliche parkmöglichkeiten befinden sich in...
2,1684613,we guarantee the return of amount paid for the...,wir garantieren die zurückzahlung der für die ...
3,2757674,that question was put to the citizens of switz...,that question was put to the citizens of switz...
4,3663922,we welcome that because we the united states n...,we welcome that because we the united states n...


In [71]:
import nltk
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [72]:

nltk.download('stopwords')

from nltk.corpus import stopwords

german_stop_words = stopwords.words('german')
english_stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
chunk_size = 500000  # Adjust this based on available memory
max_seq_length = 50  # Adjust based on data analysis
output_dir = '/content/data1117/preprocessed_chunks_training/'
os.makedirs(output_dir, exist_ok=True)

# Initialize tokenizers
stemmer = PorterStemmer()
english_tokenizer = Tokenizer()
german_tokenizer = Tokenizer()

# Split the DataFrame into chunks manually
num_chunks = len(train_df) // chunk_size + (1 if len(train_df) % chunk_size != 0 else 0)

# Step 1: Fit Tokenizers Across Chunks
for i in range(num_chunks):
    print("num_chunk----->", i)
    chunk = train_df.iloc[i * chunk_size:(i + 1) * chunk_size]
    chunk['german_tokens'] = chunk['German'].apply(word_tokenize)
    chunk['english_tokens'] = chunk['English'].apply(word_tokenize)
    chunk['german_tokens'] = chunk['german_tokens'].apply(lambda tokens: [token for token in tokens if token not in german_stop_words])
    chunk['english_tokens'] = chunk['english_tokens'].apply(lambda tokens: [token for token in tokens if token not in english_stop_words])

    chunk['german_tokens'] = chunk['german_tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    chunk['english_tokens'] = chunk['english_tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    chunk['German'] = chunk['german_tokens'].apply(lambda tokens: ' '.join(tokens))
    chunk['English'] = chunk['english_tokens'].apply(lambda tokens: ' '.join(tokens))

    # Update tokenizers
    english_tokenizer.fit_on_texts(chunk['English'])
    german_tokenizer.fit_on_texts(chunk['German'])

    # Convert to sequences
    english_sequences = english_tokenizer.texts_to_sequences(chunk['English'])
    german_sequences = german_tokenizer.texts_to_sequences(chunk['German'])

    # Pad sequences
    english_padded = pad_sequences(english_sequences, maxlen=max_seq_length, padding='post')
    german_padded = pad_sequences(german_sequences, maxlen=max_seq_length, padding='post')

    # Save each chunk
    np.save(os.path.join(output_dir, f'english_chunk_{i}.npy'), english_padded)
    np.save(os.path.join(output_dir, f'german_chunk_{i}.npy'), german_padded)

print("Data processing in chunks completed. Tokenized sequences are saved to disk.")


num_chunk-----> 0


<ipython-input-73-be82efb5eff6>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['german_tokens'] = chunk['German'].apply(word_tokenize)
<ipython-input-73-be82efb5eff6>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['english_tokens'] = chunk['English'].apply(word_tokenize)
<ipython-input-73-be82efb5eff6>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

Data processing in chunks completed. Tokenized sequences are saved to disk.


In [74]:
chunk_size = 500000  # Adjust this based on available memory
max_seq_length = 50  # Adjust based on data analysis
output_dir = '/content/data1117/preprocessed_chunks_holdout/'
os.makedirs(output_dir, exist_ok=True)

# Initialize tokenizers
stemmer = PorterStemmer()
english_tokenizer = Tokenizer()
german_tokenizer = Tokenizer()

# Split the DataFrame into chunks manually
num_chunks = len(hold_out_df) // chunk_size + (1 if len(hold_out_df) % chunk_size != 0 else 0)

# Step 1: Fit Tokenizers Across Chunks
for i in range(num_chunks):
    print("num_chunk----->", i)
    chunk = hold_out_df.iloc[i * chunk_size:(i + 1) * chunk_size]
    chunk['german_tokens'] = chunk['German'].apply(word_tokenize)
    chunk['english_tokens'] = chunk['English'].apply(word_tokenize)
    chunk['german_tokens'] = chunk['german_tokens'].apply(lambda tokens: [token for token in tokens if token not in german_stop_words])
    chunk['english_tokens'] = chunk['english_tokens'].apply(lambda tokens: [token for token in tokens if token not in english_stop_words])

    chunk['german_tokens'] = chunk['german_tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    chunk['english_tokens'] = chunk['english_tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    chunk['German'] = chunk['german_tokens'].apply(lambda tokens: ' '.join(tokens))
    chunk['English'] = chunk['english_tokens'].apply(lambda tokens: ' '.join(tokens))

    # Update tokenizers
    english_tokenizer.fit_on_texts(chunk['English'])
    german_tokenizer.fit_on_texts(chunk['German'])

    # Convert to sequences
    english_sequences = english_tokenizer.texts_to_sequences(chunk['English'])
    german_sequences = german_tokenizer.texts_to_sequences(chunk['German'])

    # Pad sequences
    english_padded = pad_sequences(english_sequences, maxlen=max_seq_length, padding='post')
    german_padded = pad_sequences(german_sequences, maxlen=max_seq_length, padding='post')

    # Save each chunk
    np.save(os.path.join(output_dir, f'english_chunk_{i}.npy'), english_padded)
    np.save(os.path.join(output_dir, f'german_chunk_{i}.npy'), german_padded)

print("Data processing in chunks completed. Tokenized sequences are saved to disk.")


num_chunk-----> 0


<ipython-input-74-ff2f5c8d65ae>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['german_tokens'] = chunk['German'].apply(word_tokenize)
<ipython-input-74-ff2f5c8d65ae>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['english_tokens'] = chunk['English'].apply(word_tokenize)
<ipython-input-74-ff2f5c8d65ae>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

Data processing in chunks completed. Tokenized sequences are saved to disk.


In [76]:
# Initialize lists to store all data
english_data = []
german_data = []
output_dir = '/content/data1117/preprocessed_chunks_training/'
# Iterate through all saved chunks
num_chunks = len([name for name in os.listdir(output_dir) if name.startswith('english_chunk_')])

for i in range(num_chunks):
    # Load the English and German chunks
    english_chunk = np.load(os.path.join(output_dir, f'english_chunk_{i}.npy'))
    german_chunk = np.load(os.path.join(output_dir, f'german_chunk_{i}.npy'))

    # Append to the list
    english_data.extend(english_chunk)
    german_data.extend(german_chunk)

# Convert lists to DataFrame
preprocessed_training_data = pd.DataFrame({
    'English': english_data,
    'German': german_data
})

print("Combined DataFrame created successfully.")
print(preprocessed_training_data.head())

Combined DataFrame created successfully.
                                             English  \
0  [3983, 530, 56, 19, 676, 0, 0, 0, 0, 0, 0, 0, ...   
1  [76, 336, 123, 1431, 459, 0, 0, 0, 0, 0, 0, 0,...   
2  [387, 555, 529, 1249, 32, 0, 0, 0, 0, 0, 0, 0,...   
3  [84, 178, 242, 1480, 0, 0, 0, 0, 0, 0, 0, 0, 0...   
4  [282, 177, 8, 14, 790, 2443, 1504, 93, 22, 138...   

                                              German  
0  [19035, 2690, 256, 128, 1063, 0, 0, 0, 0, 0, 0...  
1  [1530, 12330, 928, 316, 3189, 0, 0, 0, 0, 0, 0...  
2  [3190, 19036, 242, 28666, 7379, 0, 0, 0, 0, 0,...  
3  [6, 124, 205, 3, 1, 208, 2, 5539, 0, 0, 0, 0, ...  
4  [10, 406, 6, 88, 10, 1, 276, 33, 51, 929, 3462...  


In [77]:
# Initialize lists to store all data
english_data = []
german_data = []
output_dir = '/content/data1117/preprocessed_chunks_holdout/'
# Iterate through all saved chunks
num_chunks = len([name for name in os.listdir(output_dir) if name.startswith('english_chunk_')])

for i in range(num_chunks):
    # Load the English and German chunks
    english_chunk = np.load(os.path.join(output_dir, f'english_chunk_{i}.npy'))
    german_chunk = np.load(os.path.join(output_dir, f'german_chunk_{i}.npy'))

    # Append to the list
    english_data.extend(english_chunk)
    german_data.extend(german_chunk)

# Convert lists to DataFrame
preprocessed_holdout_data = pd.DataFrame({
    'English': english_data,
    'German': german_data
})

print("Combined DataFrame created successfully.")
print(preprocessed_holdout_data.head())

Combined DataFrame created successfully.
                                             English  \
0  [10, 40, 199, 415, 42, 508, 0, 0, 0, 0, 0, 0, ...   
1  [111, 2, 2432, 2111, 2557, 798, 1051, 1512, 20...   
2  [3630, 11267, 1035, 16200, 3398, 839, 3631, 16...   
3  [547, 56, 57, 22, 1302, 1292, 9013, 2745, 1473...   
4  [347, 738, 766, 2279, 635, 428, 311, 1941, 140...   

                                              German  
0  [1, 30, 23, 563, 3, 191, 1, 605, 2, 12, 88, 68...  
1  [5053, 7483, 6936, 9035, 10181, 1278, 0, 0, 0,...  
2  [664, 5054, 10182, 26879, 17785, 5659, 3905, 1...  
3  [13, 511, 6, 190, 139, 72, 1516, 48, 1615, 8, ...  
4  [1, 1429, 1, 2377, 4, 5660, 2, 1, 1343, 1, 422...  


In [78]:
preprocessed_training_data.to_parquet('/content/drive/MyDrive/capstone_preprocess_training.parquet')
preprocessed_holdout_data.to_parquet('/content/drive/MyDrive/capstone_preprocess_holdout.parquet')

In [79]:
train_df = pd.read_parquet('/content/drive/MyDrive/capstone_preprocess_training.parquet')
hold_out_df = pd.read_parquet('/content/drive/MyDrive/capstone_preprocess_holdout.parquet')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np

# Step 1: Custom Dataset Class
class SequenceDataset(Dataset):
    def __init__(self, df, input_col, target_col, input_max_len=None, target_max_len=None):
        self.input_sequences = self.pad_sequences(df[input_col].tolist(), max_length=input_max_len)
        self.target_sequences = self.pad_sequences(df[target_col].tolist(), max_length=target_max_len)

    def pad_sequences(self, sequences, max_length=None):
        valid_sequences = [seq for seq in sequences if len(seq) > 0]
        if max_length is None:
            max_length = max(len(seq) for seq in valid_sequences)
        return np.array(
            valid_sequences, dtype=np.int64
        )

    def __len__(self):
        return len(self.input_sequences)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.input_sequences[idx], dtype=torch.long),
            torch.tensor(self.target_sequences[idx], dtype=torch.long),
        )

# Load and preprocess data
# Assume df is a Pandas DataFrame with "English" and "German" columns
input_max_len = 50  # Set maximum sequence lengths
target_max_len = 50
train_dataset = SequenceDataset(train_df, input_col="English", target_col="German", input_max_len=input_max_len, target_max_len=target_max_len)
hold_out_dataset = SequenceDataset(hold_out_df, input_col="English", target_col="German", input_max_len=input_max_len, target_max_len=target_max_len)

# DataLoader
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
holdout_dataloader = DataLoader(hold_out_dataset, batch_size=batch_size, shuffle=True)

# Step 2: Define the RNN Model
class SimpleRNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super(SimpleRNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h_0 = torch.zeros(1, x.size(0), self.hidden_dim).to(x.device)  # Initialize hidden state
        output, _ = self.rnn(x, h_0)  # RNN forward pass
        output = self.fc(output)  # Fully connected layer
        return output

# Model parameters
input_dim = train_df["English"].apply(lambda x: max(x)).max() + 1  # Vocabulary size of English
output_dim = train_df["German"].apply(lambda x: max(x)).max() + 1  # Vocabulary size of German
hidden_dim = 128

# Initialize model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = SimpleRNN(input_dim, output_dim, hidden_dim).to(device)
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss expects raw logits
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 3: Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for inputs, targets in train_dataloader:
        # One-hot encode inputs
        inputs_one_hot = torch.nn.functional.one_hot(inputs, num_classes=input_dim).float().to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs_one_hot)

        # Reshape outputs and targets for loss computation
        outputs = outputs.view(-1, output_dim)  # Flatten for CrossEntropyLoss
        targets = targets.view(-1)  # Flatten targets

        # Compute loss
        loss = criterion(outputs, targets)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_dataloader)}")

# Step 4: Evaluation (optional)
model.eval()
with torch.no_grad():
    for inputs, targets in holdout_dataloader:
        inputs_one_hot = torch.nn.functional.one_hot(inputs, num_classes=input_dim).float().to(device)
        outputs = model(inputs_one_hot)
        print("Sample prediction:", torch.argmax(outputs, dim=2)[0].cpu().numpy())
        break


cuda
Epoch 1/10, Loss: 3.189885659693138


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Define a function to compute BLEU score
def compute_bleu_score(reference, hypothesis):
    """
    Compute BLEU score for a single pair of reference and hypothesis sequences.
    :param reference: List of integers (ground truth sequence)
    :param hypothesis: List of integers (predicted sequence)
    :return: BLEU score
    """
    smoothing = SmoothingFunction().method1
    return sentence_bleu([reference], hypothesis, smoothing_function=smoothing)

# Evaluate BLEU score on a batch
model.eval()
total_bleu = 0
count = 0

with torch.no_grad():
    for inputs, targets in holdout_dataloader:
        inputs_one_hot = torch.nn.functional.one_hot(inputs, num_classes=input_dim).float().to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs_one_hot)

        # Get the predicted sequences
        predictions = torch.argmax(outputs, dim=2)  # Shape: [batch_size, seq_length]

        # Compute BLEU for each sequence in the batch
        for i in range(predictions.size(0)):
            reference = targets[i].cpu().tolist()
            hypothesis = predictions[i].cpu().tolist()

            # Remove padding tokens (0s)
            reference = [token for token in reference if token != 0]
            hypothesis = [token for token in hypothesis if token != 0]

            bleu_score = compute_bleu_score(reference, hypothesis)
            total_bleu += bleu_score
            count += 1

# Compute the average BLEU score for the dataset
average_bleu = total_bleu / count
print(f"Average BLEU Score: {average_bleu:.4f}")
